<a href="https://colab.research.google.com/github/mohamedaliELfeky/Spelling-Corrector/blob/main/Word_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports 

In [ ]:
import os
import re
from collections import Counter, defaultdict

import numpy as np
import pandas as pd

from requests import get
from termcolor import colored


In [ ]:
# load bigdata

p = 'https://norvig.com/big.txt'
page_rquest = get(p).text
with open('big.txt', 'w') as f:
    f.writelines(page_rquest)


In [ ]:
# load common errors

misspelling_file = get("https://norvig.com/ngrams/spell-errors.txt").text
misspelling_file = misspelling_file.lower()
with open("misspelling.txt", 'w') as f:
    f.writelines(misspelling_file)



In [ ]:
# load test data

testCases = {'spell-testset1.txt':"https://norvig.com/spell-testset1.txt",
             'spell-testset2.txt':"https://norvig.com/spell-testset1.txt"}

for i in testCases:
    with open(i, 'w') as f:
        f.writelines(get(testCases[i]).text)




# Config

In [ ]:
LABLCE_SMOOTHING = 1e-5

VOCAB_PATH = "big.txt"


# Helper Functions 

## Multiply dictionaries of probability

In [ ]:
def mul_dict(x:dict, y:dict):
    """
    Multiply probability of 2 dictionary
    Input:
        x(dict): all vocab probabilty that similar to the word that need correction
        y(dict): all vocab probabilty that apeared in the same context

    Output:
        z(dict): sorted dictionary of the liklihood probability
    """
    for i in x:
        if i in y:
            x[i] *= y[i]
        else:
            # if the word didn't appear in the context we will reduce its probabilty
            x[i] *= LABLCE_SMOOTHING

    z = list(sorted(x.items(), key=lambda x: x[1]))
    return dict(reversed(z))
                    

## Generate regex expression

In [ ]:
def generate_exp(w1=None, w2=None):
    """
    Generates Regex expressions
    Input
        w1: starting word for the search
        w2: ending word for the search
    Output
        pattern: regex pattern 
    """
    if w1 and w2:
        return rf"{w1} \w+ {w2}"
    elif w1:
        return rf'{w1} \w+\b'
    elif w2:
        return rf'\w+ {w2}'
    
    return r'[a-zA-Z]+'

## reading and processing data

In [ ]:
def read_misspelled(file_path='misspelling.txt'):
    """
    Read the misspelled file
    Input:
        file_path(str): string contains the path to the file
    Output:
        corrections_dict(dict): contain the correct words and the misspelled 
    """
    file_text = None
    corrections_dict = dict()
    with open(file_path, 'r') as f:
        file_text = f.readlines()

    for line in file_text:
        corrct, miss_l = line.split(':')

        corrections_dict[corrct] = list(map(lambda x:x.strip().lower(), miss_l.split(',')))

    return corrections_dict


def read_datafile(file_name):
    """
    Read text file and return lower case letter 
    Input:
        file_name: path to the file with data or the dictionary of the words
    Output:
        textfile: lower case string with the content of the file_name
    """
    
    textfile = str()

    with open(file_name) as f:
        textfile = "".join(f.readlines())
        
    textfile = textfile.lower()

    return textfile



def process_data(textfile):
    """
    Return list of words in the string 
    Input: 
        textfile: lower case string with the content of the file_name
    Output: 
        words: a list containing all the words in the corpus (text file you read) in lower case. 
    """
    words = [] 
    
    words = re.findall(generate_exp(), textfile)
    
    return words


# Count the words occurance in the Docs
def get_count(word_l):
    '''
    Get the Count for each word in the list
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''
    
    word_count_dict = {}

    word_count_dict = Counter(word_l)

    return word_count_dict


# Get the probability of the word in the docs
def get_probs(word_count_dict):
    '''
    Get probability of every word in the dictionary
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {}

    total_words = sum(word_count_dict.values())
    probs = dict(map(lambda x:(x[0],x[1]/total_words), word_count_dict.items()))
    
    return probs

def get_aphaUnigram(vocab):
    """
    Get the count of each alphabet in the vocab
    Input:
        vocab(set): set of words in the big data
    Output:
        alpha_dict(dict): dictionary contain count of each character
    """
    alpha_dict = defaultdict(int)
    for word in vocab:

        for alpha in word:
            alpha_dict[alpha] += 1

    return alpha_dict

def get_aphaBigram(vocab):
    """
    Get the count of each consecutive alphabet in the vocab
    Input:
        vocab(set): set of words in the big data
    Output:
        alpha_bigram(dict): dictionary contain count of each consecutive character
    """
    alpha_bigram = defaultdict(int)

    for correct in vocab:
        correct = "#" + correct
        for word in range(len(correct) - 1):
            token1 = correct[word]
            token2 = correct[word + 1]
            alpha_bigram[tuple((token1, token2))] += 1

    return alpha_bigram

def get_min_word(corrected, wrong_word):
    """
    Get the distance between any 2 words
    Input:
        corrected(dict): dictionary of words with its probability 
        wrong_word(str): the word to check the distance from
    Output:
        selected(str): the word with smallest distance from wrong_word
    """
    selected = str()
    min_edits = 100
    corrected = dict(reversed(list(sorted(corrected.items(), key=lambda x: x[1]))))
    for target, prob in corrected.items():
        _, med = min_edit_distance(wrong_word, target) # (will have edits in the comming edits)

        if med < min_edits:
            selected = target
        
    return selected

## norvig word creation

In [ ]:
# delete character from a word
def delete_letter(word, verbose=False):
    '''
    Delete character from word at each position
    Input:
        word(str): the string/word for which will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        delete_l(list): a list of all possible strings obtained by deleting 1 character from word
    '''
    
    delete_l = []
    split_l = []
    
    split_l = [(word[:w], word[w:]) for w in range(len(word))]
    delete_l = [R + L[1:] for R, L in split_l if L]
    

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l


# switch each neighboring characters
def switch_letter(word, verbose=False):
    '''
    Switch any two consecutive charater
    Input:
        word: the string/word for which will generate all possible words 
                in the vocabulary which have 1 switch characters
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    ''' 
    
    switch_l = []
    split_l = []
    
    split_l = [(word[:w], word[w:]) for w in range(len(word))]
    
    switch_l = [R + L[1] + L[0] + L[2:] for R, L in split_l if len(L) > 1]
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l



# replace a charcter 
def replace_letter(word, verbose=False):
    '''
    Replace any two consecutive charater
    Input:
        word: the string/word for which will generate all possible words 
                in the vocabulary which have 1 replaced character
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    
    split_l = [(word[:w], word[w:]) for w in range(len(word))]
    replace_l = [R + l + L[1:] for R, L in split_l for l in letters if L]
    replace_set = set(replace_l)
    
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

# insert character in the word 
def insert_letter(word, verbose=False):
    '''
     insert character at each position in the word
    Input:
        word: the string/word for which will generate all possible words 
                in the vocabulary which have 1 switch characters
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
    split_l = [(word[:w], word[w:]) for w in range(len(word) +  1)]
    insert_l = [R + l + L for R, L in split_l for l in letters]
    
    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l


# get the minimum distance between two words

def edit_one_letter(word, allow_switches = True):
    """
    Generate a list of all possible combination resulted from one edit
    Input:
        word: the string/word for which we will generate all possible words that are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit.
    """
    
    edit_one_set = list()
    
    edit_one_set = insert_letter(word) + delete_letter(word) + replace_letter(word)
    if allow_switches:
        edit_one_set += switch_letter(word)
        
    while edit_one_set.count(word):
        edit_one_set.remove(word)

    return set(edit_one_set)


def edit_two_letters(word, allow_switches = True):
    '''
    Generate a list of all possible combination resulted from two edit
    Input:
        word: the string/word for which we will generate all possible words 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''
    
    edit_two_set = list()    
    
    step_one = edit_one_letter(word)

    for w in map(lambda x:edit_one_letter(x, allow_switches=allow_switches), step_one):
        edit_two_set += list(w)

    return set(edit_two_set)


# norvig new edtits

In [ ]:

def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Get the minimum distance between two words
    Input: 
        source: a string corresponding to the string we are starting with
        target: a string corresponding to the string we want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    
    m = len(source) 
    n = len(target) 
    
    D = np.zeros((m+1, n+1), dtype=int) 
    
    D[0,0] = 0
    
    for row in range(1,m + 1):
        D[row,0] = D[row - 1, 0] + del_cost
        
    
    for col in range(1,n + 1):
        D[0,col] = D[0,col-1] + ins_cost
        
    for row in range(1,m + 1): 
        
        
        for col in range(1,n + 1):
            
            
            r_cost = rep_cost
            
            
            if source[row - 1] is target[col - 1]:
                
                r_cost = 0
                
            D[row,col] = min(D[row - 1, col] + ins_cost,
                             D[row, col - 1] + del_cost,
                             D[row - 1,col - 1] + r_cost
                            )
          
    med = D[m, n]
    
    return D, med



def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Get dictionary with n-th bigest probability words/candidates 
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections we want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = set()
    n_best = dict()

    if word in vocab:
        suggestions.update(set([word]))

    else:
        suggestions.update(edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab) or set([word]))

    
    n_best.update(dict(list((s, probs[s]) if s in probs else (s, 0) for s in suggestions)))

    n_best = dict(list(reversed(sorted(n_best.items(), key=lambda x:x[1])))[:n])

    
    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best


def get_ngrams(textfile, w1=None, w2=None):
    """
    Get the n-gram for any word between w1 and w2
    Input:
        textfile(str): string with lower case letters
        w1(str): word usually start of the pattern
        w2(str): word usually end of the pattern
        
    Output:
        n_grams_dict: dictionary with count of the words that have the pattern
    """
    
    n_grams_dict = dict()
    
    file_pattern = re.findall(generate_exp(w1, w2), textfile)

    if not len(file_pattern) and w1 and w2:
        file_pattern = re.findall(generate_exp(w1=w1), textfile)

    file_pattern = " ".join(file_pattern).split(" ")
    count_file = get_count(file_pattern)
    

    count_file.pop(w1) if w1 and w1 in count_file else None
    count_file.pop(w2) if w2 and w2 in count_file else None
    
    n_grams_dict = get_probs(count_file)
    
    return n_grams_dict
    
    

def get_unigrams(textfile):
    """
    Get the probability for each word
    Input:
        textfile(str): string with lower case letters
    Output:
        words_prob(dict): probabilities of all the words
    """
    words_prob = dict()

    words_l = process_data(textfile)

    words_cont = get_count(words_l)

    words_prob = get_probs(words_cont)

    return words_prob, set(words_l), words_cont





# noisy channle

In [ ]:
def check_ops(correct, noisy, ops):
    """
    Get which characters have a ops done on it
    Input:
        correct(str): the correct word
        noisy(str): the noisy word that have the operation done
        ops: the operation
    Output:
        tupel(c1, c2): tuple of the two characters that have the operation
    """

    min_len = min(len(correct), len(noisy))

    for i in range(min_len):

        if correct[i] != noisy[i]:
            if i == 0:
                return ('#', correct[i]) if ops == 'del' else  ('#' , noisy[i])
            
            return (correct[i - 1], correct[i]) if ops == 'del' else (correct[i - 1], noisy[i])

    return (correct[-2], correct[-1]) if ops == 'del' else (correct[-1], noisy[-1])


In [ ]:
def get_editOps(correct, noisy):
    """
    Get the operation done over the word correct to get the word noisy
    Input:
        correct(str): the original word
        noisy(str): the word with error
    Output:
        tuple(ops, c1, c2): tuple have the operatoin and the characters that the operation done over it
    """

    if correct == noisy:
        return "same", (-1, -1)

    if len(correct) < len(noisy):
        return "ins", check_ops(correct, noisy, 'ins')

    elif len(correct) > len(noisy):
        return "del", check_ops(correct, noisy, 'del')

    
    return tuple(('rep', (noisy[i], correct[i])) for i in range(len(noisy)) if noisy[i] != correct[i])[0]



In [ ]:
def build_table():
    """
    Build tables for each operation {insertion, deletion, replacement}
    Output:
        fram(DataFrame): with count for operation that the table have
    """
    alphabit = '#abcdefghijklmnopqrstuvwxyz'

    fram = pd.DataFrame(columns=list(alphabit)[1:], index=list(alphabit))
    fram.fillna(0, inplace=True)

    return fram

def save_tables(tables):
    """
    saves the tables in a csv to faster usge
    Input:
        tables(list): list of data frames that have the oreder of 'insert', 'delete', 'replace'
    """
    
    names = ['insert', 'delete', 'replace']

    for num in range(len(tables)):
        tables[num].to_csv(f"{names[num]}.csv")

In [ ]:
# get the misspelled words and its correction as a dictionary
misspelling_dict = read_misspelled()

# build the three error operation count table
InsOps = build_table()
DelOps = build_table()
repOps = build_table() 


# fill the tables with the count of operations
for correct_word in misspelling_dict:
    for missed in misspelling_dict[correct_word]:
        
        # cleaning the words to easier processing
        missed = re.findall('[a-zA-Z]+', missed)

        cleaned_word = re.findall('[a-zA-Z]+', correct_word)

        # if the missed and cleaned_word not exist we will continue
        if not missed:
            continue
        if not cleaned_word:
            break

        cleaned_word = cleaned_word[0]
        missed = missed[0]

        # get the operation and the characters
        (ops, (x, y)) = get_editOps(cleaned_word, missed)
    
        
        # store the count of operation
        if ops == 'ins':
            InsOps[y][x] += 1
        elif ops == 'del':
            DelOps[y][x] += 1
        elif ops == 'rep':
            repOps[y][x] += 1

# save the tables
save_tables([InsOps, DelOps, repOps])

In [ ]:
def get_count_unialpha(x):
    """
    Get the count of single characters or space
    Input:
        x(str): characters
    Output:
        int: of the count of that characters
    """
    if x == '#':
        return len(vocab)
    
    return count_unialpha[x]

In [ ]:
def getProbabilityOfWord(word,noisy):
    """
    Get the probability of each operation for each 2 characters
    Input:
        word(str): the correct word
        noisy(str): the noisy word
    Output:
        probability(float): probability of each operation
    """

    operation, (x, y) = get_editOps(word,noisy)
    if operation == 'ins':
        probability = InsOps[y][x]/(get_count_unialpha(x) + LABLCE_SMOOTHING)
    elif operation == 'del':
        probability = DelOps[y][x]/(count_bialpha[x,y] + LABLCE_SMOOTHING)
    elif operation == 'rep':
        probability = repOps[y][x]/(get_count_unialpha(y) + LABLCE_SMOOTHING)
    elif operation == 'same':
        probability = 1
    return probability

In [ ]:
def get_noisy(recommended, corrections_word):
    """
    Get the probability of the noist channel model for the 2 words
    Input:
        recommended(dict): dictionary of each generated word and its probability
        corrections_word(str): the noisy word we want to get the correct from
    Output:
        options(dict): the recommernded words that could have
                        the noisy word generated from with probabilities
    """
    options = {} # Dictionary
    for w in recommended:
        prob_xw = getProbabilityOfWord(w,corrections_word)  # P(x|w)
        #prob_w = words_prob[w]  # P(w)
        result = prob_xw * recommended[w]      # P(x|w) . P(w)
        options[w] = result                  # Store the result in the dict


    return options 
            
        

# Main

In [ ]:
# get correction using noisy channel model
def correction(wrong_word, n=5, is_sentance=False):
    """
    Get the Correction for the wrong_word
    Input:
        wrong_word(str): the misspelled word
        n(int): the number of candidates we want to process
        is_sentance: check if we working on sentance of just one word
    output:
        recommended(str/list): it's the list of corrected word or just the word with the biggest probability
    """
    # get all possible words with n-th heighst probabilities if the word not in the vocab 
    recommended = get_corrections(wrong_word, words_prob, vocab, n)
    # get the noisy channel recommendation for single word
    recommended = get_noisy(recommended, wrong_word)
    
    if is_sentance:
        return recommended
    
    return max(recommended, key=recommended.get)  # Likely word is the word with the maximum probability 
    
    


if __name__ == '__main__':
    
    # take the sentance from the user
    sentance = input("Enter your sentance: ").strip().lower()
    
    # read the file and return string with lower case letters
    text_file = read_datafile(VOCAB_PATH)

    # unigram function use other functions to get dictionary of probabilties for all words
    # and set of all the vocabulary in the file
    words_prob, vocab, WORDS = get_unigrams(text_file)

    # noisy channel dictionaries
    count_unialpha = get_aphaUnigram(vocab)
    count_bialpha = get_aphaBigram(vocab)

    # split the input to itrate on
    corrections_words = sentance.split(" ")

    # create new list of the sentance with the corrections
    new_sentance = [None] * len(corrections_words)

    # looping over the sentance
    for word in range(len(corrections_words)) :

        # get all possible words with n-th heighst probabilities if the word not in the vocab 
        recommended = correction(corrections_words[word], n=5, is_sentance=True)
        
        # conditional probability goes from here
        ngrams_dict = []
        
        max_prob = 0
        min_edits = 100
        selected = str()

        # geting bi-gram and tri gram 
        if word == 0 and len(corrections_words) > 1:
            # if the word in the beging of the sentance will get all possible words usually come with the second one
            ngrams_dict = get_ngrams(text_file, w2=corrections_words[word + 1])

        elif word + 1 == len(corrections_words) and word > 0:
            # if the word in the ending of the sentance will get all possible words usually come with before it
            ngrams_dict = get_ngrams(text_file, w1=corrections_words[word - 1])

        elif word >= 1:
            # getting the tri gram
            ngrams_dict = get_ngrams(text_file, w1=corrections_words[word - 1],
                                    w2=corrections_words[word + 1])
        


        # muliply the probabilties of the the possible words and the conditional probability words
        # it's similar to maximum likelihood
        recommended = mul_dict(recommended, ngrams_dict)

        selected = max(recommended, key=recommended.get)
        
        
        # selected = get_min_word(recommended)

        new_sentance[word] = selected
            
        

    for w in range(len(new_sentance)):
        if new_sentance[w] == corrections_words[w]:
            print(colored(new_sentance[w], 'green'), end=" ")

        else:
            print(colored(new_sentance[w], 'red'), end=" ")
    print()



Enter your sentance: Hi whataa ara wou
hi what are you 


# Evaluation

In [ ]:
# def unit_tests():
#     assert correction('speling') == 'spelling'              # insert
#     assert correction('korrectud') == 'corrected'           # replace 2
#     assert correction('bycycle') == 'bicycle'               # replace
#     assert correction('inconvient') == 'inconvenient'       # insert 2
#     assert correction('arrainged') == 'arranged'            # delete
#     assert correction('peotry') =='poetry'                  # transpose
#     assert correction('peotryy') =='poetry'                 # transpose + delete
#     assert correction('word') == 'word'                     # known
#     assert correction('quintessential') == 'quintessential' # unknown
#     assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
#     assert Counter(words('This is a test. 123; A TEST this is.')) == (
#            Counter({'123': 1, 'a': 2, 'is': 2, 'test': 2, 'this': 2}))
#     assert len(WORDS) == 32192
#     assert sum(WORDS.values()) == 1115504
#     assert WORDS.most_common(10) == [
#      ('the', 79808),
#      ('of', 40024),
#      ('and', 38311),
#      ('to', 28765),
#      ('in', 22020),
#      ('a', 21124),
#      ('that', 12512),
#      ('he', 12401),
#      ('was', 11410),
#      ('it', 10681)]
#     assert WORDS['the'] == 79808
#     assert P('quintessential') == 0
#     assert 0.07 < P('the') < 0.08
#     return 'unit_tests pass'
unknown_words = []
def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown_words.append(right if right not in WORDS else None)
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

# print(unit_tests())
spelltest(Testset(open('spell-testset1.txt')), True) # Development set
spelltest(Testset(open('spell-testset2.txt'))) # Final test set

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


correction(contende) => contended (9); expected contented (13)
correction(contended) => contended (9); expected contented (13)
correction(proplen) => people (899); expected problem (76)
correction(guic) => music (56); expected juice (5)
correction(jucie) => julie (71); expected juice (5)
correction(juse) => just (767); expected juice (5)
correction(compair) => complain (14); expected compare (29)
correction(transportibility) => transportibility (0); expected transportability (0)
correction(miniscule) => miniscule (0); expected minuscule (0)
correction(poartry) => party (298); expected poetry (10)
correction(stanerdizing) => stanerdizing (0); expected standardizing (0)
correction(biscutes) => disputes (27); expected biscuits (8)
correction(receite) => receive (95); expected receipt (13)
correction(reciet) => recite (4); expected receipt (13)
correction(remined) => remained (231); expected remind (9)
correction(annt) => anna (294); expected aunt (52)
correction(ther) => their (2955); exp

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


81% of 270 correct (6% unknown) at 21 words per second 


# Resources
https://web.stanford.edu/~jurafsky/slp3/B.pdf